In [ ]:
import pymysql
import datetime
from datetime import timedelta
import datetime
from datetime import timedelta

# Configuración BD
DB_HOST = "10.254.33.138"
DB_USER = "compensaciones_rrhh"
DB_PASSWORD = "_Cramercomp2025_"
DB_NAME = "rrhh_app"


In [ ]:
def calcular_fecha_alerta(empleado):
    """
    Calcula la fecha de alerta según el tipo de contrato y status
    """
    
    tipo_contrato = (empleado.get("contract_type") or "").lower()
    status = (empleado.get("status") or "").lower()
    metodo_pago = (empleado.get("payment_method") or "").lower()
    fecha_activacion = empleado.get("active_since")
    termino_primer_plazo = empleado.get("contract_finishing_date_1")
    termino_segundo_plazo = empleado.get("contract_finishing_date_2")
    
    # Solo considerar empleados activos con transferencia bancaria o tipo de contrato fijo
    if status != "activo" or metodo_pago != "transferencia bancaria" or tipo_contrato == "indefinido":
        return None
    
    try:
        if fecha_activacion:
            fecha_activacion = datetime.datetime.strptime(fecha_activacion, "%Y-%m-%d")
            primer_plazo = datetime.datetime.strptime(termino_primer_plazo, "%Y-%m-%d") if termino_primer_plazo else None
            segundo_plazo = datetime.datetime.strptime(termino_segundo_plazo, "%Y-%m-%d") if termino_segundo_plazo else None
        else:
            return None
        
        fecha_alerta, motivo, tipo_alerta = None, None, None
        
        # Primera alerta → paso a segundo plazo
        if tipo_contrato == "fijo" and primer_plazo is not None and segundo_plazo is not None:
            fecha_alerta = fecha_activacion + timedelta(days=14)
            motivo = "Paso a segundo plazo de contratación"
            tipo_alerta = "SEGUNDO_PLAZO"
        
        # Segunda alerta → paso a indefinido  
        elif tipo_contrato == "fijo" and primer_plazo is not None and segundo_plazo is None:
            fecha_alerta = fecha_activacion + timedelta(days=76)
            motivo = "Paso a contrato indefinido"
            tipo_alerta = "INDEFINIDO"
        
        if fecha_alerta:
            return {
                "fecha_alerta": fecha_alerta.strftime("%Y-%m-%d"),
                "motivo": motivo,
                "tipo_alerta": tipo_alerta,
                "dias_desde_inicio": (datetime.datetime.now() - fecha_activacion).days,
                "requiere_accion": fecha_alerta <= datetime.datetime.now(),
                "urgente": (fecha_alerta - datetime.datetime.now()).days <= 7 if fecha_alerta > datetime.datetime.now() else True
            }
        else:
            return None
            
    except ValueError as e:
        print(f"Error procesando fechas para {empleado.get('full_name')}: {e}")
        return None

In [ ]:
print("🚀 Conectando a MySQL...")
conexion = pymysql.connect(
    host=DB_HOST,
    user=DB_USER,
    password=DB_PASSWORD,
    charset='utf8mb4'
)
cursor = conexion.cursor()

# Usar la base de datos existente
cursor.execute(f"USE {DB_NAME}")
print(f"✅ Conectado a MySQL y usando la base: {DB_NAME}")

print("🚀 Creando tabla contract_alerts...")
sql_create_alerts_table = """
CREATE TABLE IF NOT EXISTS contract_alerts (
    id INT AUTO_INCREMENT PRIMARY KEY,
    employee_id INT NOT NULL,
    -- person_id INT,
    employee_name VARCHAR(255),
    employee_rut VARCHAR(50),
    -- employee_email VARCHAR(255),
    -- employee_area_id INT,
    employee_area_name VARCHAR(255),
    employee_role VARCHAR(255),
    employee_start_date DATE,
    employee_contract_type VARCHAR(50),
    -- employee_base_wage INT,
    
    -- Información del jefe
    -- boss_id INT,
    -- boss_rut VARCHAR(50),
    boss_name VARCHAR(255),
    boss_email VARCHAR(255),
    
    -- Información de la alerta
    alert_date DATE NOT NULL,
    alert_type VARCHAR(50),
    alert_reason TEXT,
    days_since_start INT,
    requires_action BOOLEAN DEFAULT FALSE,
    is_urgent BOOLEAN DEFAULT FALSE,
    
    -- Campos de control
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP,
    processed BOOLEAN DEFAULT FALSE,
    notes TEXT,
    
    -- Índices para búsquedas rápidas
    INDEX idx_employee_id (employee_id),
    -- INDEX idx_boss_id (boss_id),
    INDEX idx_alert_date (alert_date),
    INDEX idx_requires_action (requires_action),
    INDEX idx_is_urgent (is_urgent),
    INDEX idx_processed (processed)
);
"""

cursor.execute(sql_create_alerts_table)
print("✅ Tabla 'contract_alerts' creada exitosamente")

In [ ]:
def obtener_info_jefe(id_boss, rut_boss, empleados_lista):
    """
    Busca la información del jefe en la lista de empleados
    """
    if not id_boss and not rut_boss:
        return None, None, None
    
    for empleado in empleados_lista:
        # Buscar por ID primero, luego por RUT
        if (id_boss and empleado.get("id") == id_boss) or (rut_boss and empleado.get("rut") == rut_boss):
            return (
                empleado.get("full_name"),
                empleado.get("email"),
                empleado.get("rut")
            )
    
    return None, None, None

def obtener_nombre_area(area_id, cursor):
    """
    Obtiene el nombre del área desde la tabla areas
    """
    if not area_id:
        return None
    
    try:
        cursor.execute("SELECT name FROM areas WHERE id = %s", (area_id,))
        resultado = cursor.fetchone()
        return resultado[0] if resultado else None
    except Exception as e:
        print(f"Error obteniendo área {area_id}: {e}")
        return None

In [ ]:
print("📊 Cargando empleados desde la base de datos...")

# Consulta para obtener todos los empleados necesarios
sql_empleados = """
SELECT 
    id, person_id, full_name, rut, email, area_id, name_role,
    start_date, contract_type, base_wage, id_boss, rut_boss,
    active_since, contract_finishing_date_1, contract_finishing_date_2,
    status, payment_method
FROM employees 
WHERE status = 'activo'
"""

cursor.execute(sql_empleados)
empleados_db = cursor.fetchall()

# Convertir a lista de diccionarios para facilitar el manejo
empleados_lista = []
columnas = [
    'id', 'person_id', 'full_name', 'rut', 'email', 'area_id', 'name_role',
    'start_date', 'contract_type', 'base_wage', 'id_boss', 'rut_boss',
    'active_since', 'contract_finishing_date_1', 'contract_finishing_date_2',
    'status', 'payment_method'
]

for empleado_tupla in empleados_db:
    empleado_dict = {}
    for i, columna in enumerate(columnas):
        valor = empleado_tupla[i]
        # Convertir fechas a string si es necesario
        if isinstance(valor, datetime.date):
            valor = valor.strftime("%Y-%m-%d")
        empleado_dict[columna] = valor
    empleados_lista.append(empleado_dict)

print(f"✅ Cargados {len(empleados_lista)} empleados activos desde la BD")

In [ ]:
# %%
print("🔄 === PROCESANDO ALERTAS DE CONTRATOS ===")

# Limpiar alertas anteriores (opcional - descomenta si quieres empezar limpio cada vez)
# cursor.execute("DELETE FROM contract_alerts WHERE processed = FALSE")
# print("🗑️ Alertas anteriores eliminadas")

alertas_insertadas = 0
empleados_procesados = 0
errores = 0

for empleado in empleados_lista:
    empleados_procesados += 1
    
    # Calcular si este empleado necesita alerta
    alerta = calcular_fecha_alerta(empleado)
    
    if alerta:
        try:
            # Obtener información del jefe
            boss_name, boss_email, boss_rut_real = obtener_info_jefe(
                empleado.get("id_boss"), 
                empleado.get("rut_boss"), 
                empleados_lista
            )
            
            # Obtener nombre del área
            area_name = obtener_nombre_area(empleado.get("area_id"), cursor)
            
            # Preparar datos para insertar
            sql_insert = """
            INSERT INTO contract_alerts (
                employee_id, employee_name, employee_rut, employee_area_name, employee_role, employee_start_date, 
                employee_contract_type,
                boss_name, boss_email,
                alert_date, alert_type, alert_reason, days_since_start,
                requires_action, is_urgent
            ) VALUES (
                %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s
            )
            """
            
            # Ejecutar inserción
            cursor.execute(sql_insert, (
                empleado["id"],                    # employee_id
                #empleado["person_id"],             # person_id
                empleado["full_name"],             # employee_name
                empleado["rut"],                   # employee_rut
                #empleado["email"],                 # employee_email
                #empleado["area_id"],               # employee_area_id
                area_name,                         # employee_area_name
                empleado["name_role"],             # employee_role
                empleado["start_date"],            # employee_start_date
                empleado["contract_type"],         # employee_contract_type
                #empleado["base_wage"],             # employee_base_wage
                #empleado["id_boss"],               # boss_id
                #boss_rut_real or empleado["rut_boss"],  # boss_rut
                boss_name,                         # boss_name
                boss_email,                        # boss_email
                alerta["fecha_alerta"],            # alert_date
                alerta["tipo_alerta"],             # alert_type
                alerta["motivo"],                  # alert_reason
                alerta["dias_desde_inicio"],       # days_since_start
                alerta["requiere_accion"],         # requires_action
                alerta["urgente"]                  # is_urgent
            ))
            
            alertas_insertadas += 1
            
            # Mostrar progreso cada 50 empleados
            if empleados_procesados % 50 == 0:
                print(f"📊 Procesados: {empleados_procesados}/{len(empleados_lista)} empleados...")
                
        except Exception as e:
            errores += 1
            print(f"❌ Error insertando alerta para {empleado.get('full_name', 'N/A')}: {e}")

# Confirmar cambios en la base de datos
conexion.commit()

print(f"""
✅ === PROCESO COMPLETADO ===
👥 Empleados procesados: {empleados_procesados}
🚨 Alertas generadas: {alertas_insertadas}
❌ Errores: {errores}
💾 Cambios guardados en la base de datos
""")

# Verificación rápida
cursor.execute("SELECT COUNT(*) FROM contract_alerts WHERE processed = FALSE")
total_alertas = cursor.fetchone()[0]
print(f"🔍 Total alertas activas en BD: {total_alertas}")

In [ ]:
def calcular_fecha_alerta(empleado):
    """
    Calcula la fecha de alerta según el tipo de contrato y status
    """
    tipo_contrato = empleado.get("contract_type", "").lower()
    status = empleado.get("status", "").lower()
    metodo_pago = empleado.get("payment_method", "").lower()
    fecha_activacion = empleado.get("active_since")
    termino_primer_plazo = empleado.get("contract_finishing_date_1")
    termino_segundo_plazo = empleado.get("contract_finishing_date_2")
    
    # Solo considerar empleados activos con transferencia bancaria
    if status != "activo" or metodo_pago != "transferencia bancaria" or tipo_contrato == "indefinido":
        return None
    
    try:
        if fecha_activacion:
            fecha_activacion = datetime.datetime.strptime(fecha_activacion, "%Y-%m-%d")
            primer_plazo = datetime.datetime.strptime(termino_primer_plazo, "%Y-%m-%d") if termino_primer_plazo else None
            segundo_plazo = datetime.datetime.strptime(termino_segundo_plazo, "%Y-%m-%d") if termino_segundo_plazo else None
        else:
            return None
        
        fecha_alerta = None
        motivo = None
        tipo_alerta = None
        
        # Primera alerta → paso a segundo plazo
        if tipo_contrato == "fijo" and primer_plazo is not None and segundo_plazo is not None:
            fecha_alerta = fecha_activacion + timedelta(days=14)
            motivo = "Paso a segundo plazo de contratación"
            tipo_alerta = "SEGUNDO_PLAZO"
        
        # Segunda alerta → paso a indefinido  
        elif tipo_contrato == "fijo" and primer_plazo is not None and segundo_plazo is None:
            fecha_alerta = fecha_activacion + timedelta(days=76)
            motivo = "Paso a contrato indefinido"
            tipo_alerta = "INDEFINIDO"
        
        if fecha_alerta:
            return {
                "fecha_alerta": fecha_alerta.strftime("%Y-%m-%d"),
                "motivo": motivo,
                "tipo_alerta": tipo_alerta,
                "dias_desde_inicio": (datetime.datetime.now() - fecha_activacion).days,
                "requiere_accion": fecha_alerta <= datetime.datetime.now(),
                "urgente": (fecha_alerta - datetime.datetime.now()).days <= 7 if fecha_alerta > datetime.datetime.now() else True
            }
        else:
            return None
            
    except ValueError as e:
        print(f"Error procesando fechas para {empleado.get('full_name')}: {e}")
        return None

In [ ]:
# CONSULTAS ÚTILES PARA TU APP

print("📊 === RESUMEN DE ALERTAS ===")

# 1. Alertas urgentes (requieren acción inmediata)
cursor.execute("""
    SELECT COUNT(*) as urgentes 
    FROM contract_alerts 
    WHERE is_urgent = TRUE AND processed = FALSE
""")
urgentes = cursor.fetchone()[0]
print(f"🚨 Alertas urgentes: {urgentes}")

# 2. Alertas por procesar
cursor.execute("""
    SELECT COUNT(*) as pendientes 
    FROM contract_alerts 
    WHERE requires_action = TRUE AND processed = FALSE
""")
pendientes = cursor.fetchone()[0]
print(f"⏰ Alertas pendientes: {pendientes}")

# 3. Alertas por tipo
cursor.execute("""
    SELECT alert_type, COUNT(*) as cantidad
    FROM contract_alerts 
    WHERE processed = FALSE
    GROUP BY alert_type
""")
por_tipo = cursor.fetchall()
print("📋 Alertas por tipo:")
for tipo, cantidad in por_tipo:
    print(f"   - {tipo}: {cantidad}")

print("\n📧 === ALERTAS URGENTES DETALLADAS ===")

# 4. Obtener alertas urgentes con información completa
cursor.execute("""
    SELECT 
        employee_name, employee_rut, employee_role,
        employee_area_name, alert_date, alert_reason, days_since_start,
        boss_name, boss_email,
        DATEDIFF(alert_date, CURDATE()) as dias_hasta_alerta
    FROM contract_alerts 
    WHERE is_urgent = TRUE AND processed = FALSE
    ORDER BY alert_date ASC
    LIMIT 10
""")

alertas_urgentes = cursor.fetchall()
for alerta in alertas_urgentes:
    print(f"""
    👤 Empleado: {alerta[0]} ({alerta[1]})
    🏢 Cargo: {alerta[2]}
    📅 Fecha alerta: {alerta[4]}
    📝 Motivo: {alerta[5]}
    👔 Jefe: {alerta[7]} 
    📧 Email jefe: {alerta[8]}
    """)

print("\n🔍 === CONSULTA POR JEFE ===")

# 5. Alertas agrupadas por jefe
cursor.execute("""
    SELECT 
        boss_name, boss_email,
        COUNT(*) as total_alertas,
        SUM(CASE WHEN is_urgent = TRUE THEN 1 ELSE 0 END) as urgentes
    FROM contract_alerts 
    WHERE processed = FALSE AND boss_name IS NOT NULL
    GROUP BY boss_name, boss_email
    HAVING total_alertas > 0
    ORDER BY urgentes DESC, total_alertas DESC
""")

alertas_por_jefe = cursor.fetchall()
for jefe_info in alertas_por_jefe:
    print(f"""
    👔 Jefe: {jefe_info[0]}
    📧 Email: {jefe_info[1]}
    📊 Total alertas: {jefe_info[2]}
    """)

In [ ]:
def generar_reporte_para_jefe(boss_id, cursor):
    """
    Genera un reporte detallado para un jefe específico
    """
    sql_reporte_jefe = """
    SELECT 
        employee_name, employee_rut, employee_role,
        alert_date, alert_reason, alert_type,
        days_since_start,
        DATEDIFF(alert_date, CURDATE()) as dias_hasta_alerta,
        boss_name, boss_email
    FROM contract_alerts 
    WHERE boss_id = %s AND processed = FALSE
    ORDER BY is_urgent DESC, alert_date ASC
    """
    
    cursor.execute(sql_reporte_jefe, (boss_id,))
    alertas_jefe = cursor.fetchall()
    
    if not alertas_jefe:
        return None
    
    jefe_info = alertas_jefe[0] 
    
    reporte = {
        "jefe": {
            "nombre": jefe_info[11],
            "email": jefe_info[12]
        },
        "empleados_alerta": []
    }
    
    for alerta in alertas_jefe:
        empleado_alerta = {
            "nombre": alerta[0],
            "rut": alerta[1],
            "email": alerta[2],
            "cargo": alerta[3],
            "area": alerta[4],
            "fecha_alerta": str(alerta[5]),
            "motivo": alerta[6],
            "tipo": alerta[7],
            "dias_trabajados": alerta[8],
            "salario_base": alerta[9],
            "dias_hasta_alerta": alerta[10],
            "es_urgente": alerta[10] <= 0
        }
        reporte["empleados_alerta"].append(empleado_alerta)
    
    return reporte

def obtener_todos_los_jefes_con_alertas(cursor):
    """
    Obtiene lista de todos los jefes que tienen empleados con alertas
    """
    cursor.execute("""
        SELECT DISTINCT boss_id, boss_name, boss_email
        FROM contract_alerts 
        WHERE processed = FALSE AND boss_id IS NOT NULL
        ORDER BY boss_name
    """)
    
    return cursor.fetchall()

# Ejemplo de uso: generar reportes para todos los jefes
print("\n📧 === GENERANDO REPORTES POR JEFE ===")

jefes_con_alertas = obtener_todos_los_jefes_con_alertas(cursor)
print(f"👔 Se encontraron {len(jefes_con_alertas)} jefes con empleados que requieren renovación")

# Generar reporte para cada jefe (ejemplo con los primeros 3)
for i, (boss_id, boss_name, boss_email) in enumerate(jefes_con_alertas[:3]):
    print(f"\n--- Reporte para {boss_name} ({boss_email}) ---")
    reporte = generar_reporte_para_jefe(boss_id, cursor)
    
    if reporte:
        print(f"📊 {len(reporte['empleados_alerta'])} empleados requieren renovación:")
        for emp in reporte['empleados_alerta']:
            urgencia = "🚨 URGENTE" if emp['es_urgente'] else f"⏰ {emp['dias_hasta_alerta']} días"
            print(f"   - {emp['nombre']}: {emp['motivo']} ({urgencia})")

In [ ]:
def marcar_alerta_procesada(employee_id, notas="", cursor=None):
    """
    Marca una alerta como procesada
    """
    try:
        sql_update = """
        UPDATE contract_alerts 
        SET processed = TRUE, 
            notes = %s,
            updated_at = CURRENT_TIMESTAMP
        WHERE employee_id = %s AND processed = FALSE
        """
        cursor.execute(sql_update, (notas, employee_id))
        return True
    except Exception as e:
        print(f"Error marcando alerta como procesada: {e}")
        return False

def obtener_alertas_dashboard():
    """
    Consulta optimizada para dashboard de alertas
    """
    sql_dashboard = """
    SELECT 
        id,
        employee_name,
        employee_rut,
        employee_email,
        employee_role,
        employee_area_name,
        boss_name,
        boss_email,
        alert_date,
        alert_reason,
        days_since_start,
        DATEDIFF(alert_date, CURDATE()) as dias_hasta_vencimiento,
        CASE 
            WHEN DATEDIFF(alert_date, CURDATE()) <= 0 THEN 'VENCIDA'
            WHEN DATEDIFF(alert_date, CURDATE()) <= 7 THEN 'URGENTE'
            WHEN DATEDIFF(alert_date, CURDATE()) <= 30 THEN 'PRÓXIMA'
            ELSE 'FUTURA'
        END as prioridad,
        is_urgent,
        requires_action
    FROM contract_alerts 
    WHERE processed = FALSE
    ORDER BY 
        CASE 
            WHEN DATEDIFF(alert_date, CURDATE()) <= 0 THEN 1
            WHEN is_urgent = TRUE THEN 2
            WHEN requires_action = TRUE THEN 3
            ELSE 4
        END,
        alert_date ASC
    """
    return sql_dashboard

# Ejemplo de consulta para dashboard
print("\n🎯 === VISTA DASHBOARD ===")
cursor.execute(obtener_alertas_dashboard())
alertas_dashboard = cursor.fetchall()

print(f"📊 Total alertas activas: {len(alertas_dashboard)}")

# Mostrar las primeras 5 alertas más urgentes
print("\n🚨 Top 5 alertas más urgentes:")
for i, alerta in enumerate(alertas_dashboard[:5]):
    prioridad = alerta[12]
    emoji = "🔴" if prioridad == "VENCIDA" else "🟠" if prioridad == "URGENTE" else "🟡"
    print(f"{emoji} {alerta[1]} - {alerta[4]} - {alerta[8]} - {prioridad}")

# Cerrar conexión
cursor.close()
conexion.close()
print("\n✅ Conexión cerrada correctamente.")